In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.api as sm
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from itertools import product

def invboxcox(y,lmbda):
    if lmbda == 0:
        return(np.exp(y))
    else:
        return(np.exp(np.log(lmbda*y+1)/lmbda))

from scipy.stats import binned_statistic_2d

dan=['yellow_tripdata_2015-01.csv', 'yellow_tripdata_2015-02.csv',
     'yellow_tripdata_2015-03.csv', 'yellow_tripdata_2015-04.csv',
     'yellow_tripdata_2015-05.csv', 'yellow_tripdata_2015-06.csv',
     'yellow_tripdata_2015-07.csv', 'yellow_tripdata_2015-08.csv',
     'yellow_tripdata_2015-09.csv', 'yellow_tripdata_2015-10.csv',
     'yellow_tripdata_2015-11.csv', 'yellow_tripdata_2015-12.csv',
     'yellow_tripdata_2016-01.csv', 'yellow_tripdata_2016-02.csv',
     'yellow_tripdata_2016-03.csv', 'yellow_tripdata_2016-04.csv',
     'yellow_tripdata_2016-05.csv', 'yellow_tripdata_2016-06.csv']

def shape0(df):
    return df.shape[0]

d=pd.DataFrame(columns=[1128])

for i in dan:
    data=pd.read_csv(i, parse_dates=[1,2],
                usecols=range(7))

    mask_dist=data.trip_distance>0

    mask_time=(data.tpep_dropoff_datetime>data.tpep_pickup_datetime)

    mask_long=((data.pickup_longitude>=-74.25559) & (data.pickup_longitude<=-73.70001))

    mask_lat=((data.pickup_latitude>=40.49612) & (data.pickup_latitude<=40.91553))

    mask_pas=data.passenger_count>0

    df=data.loc[mask_dist & mask_long & mask_lat & mask_time & mask_pas]

    df.tpep_pickup_datetime=df.tpep_pickup_datetime.dt.floor('1h')

    regions=pd.read_csv('regions.csv', sep=';')

    statistic = binned_statistic_2d(df['pickup_longitude'].values, 
                                   df['pickup_latitude'].values,
                                   df.passenger_count,            #### Тут может использоваться любой столбец
                                   statistic='count', 
                                  bins=[np.union1d(regions.west, regions.east), 
                                        np.union1d(regions.south, regions.north)], 
                                  expand_binnumbers=True)
                                  
    df['region'] = (statistic[3][0] - 1) * 50 + statistic[3][1]

    agg=pd.DataFrame(index=np.unique(df.tpep_pickup_datetime), columns=regions.region.values)


    a=df[['tpep_pickup_datetime', 'region', 'VendorID']].groupby(['tpep_pickup_datetime', 'region']).apply(shape0)

    b=a.unstack()

    b.fillna(0, inplace=True)

    agg.fillna(0, inplace=True)

    c=agg+b

    c.fillna(0, inplace=True)

    c=c.astype(int)
    
    d=d.append(c[[1128]])

regions.index=regions.region
regions.drop(['region'], axis=1, inplace=True)

### Загрузил данные за 2015 год и половину 2016. Июнь 2016 будет тестовым месяцем

In [6]:
pd.read_csv('Taxi.csv', index_col=0, parse_dates=True)

,1128
2015-01-01 00:00:00,499
2015-01-01 01:00:00,449
2015-01-01 02:00:00,391
2015-01-01 03:00:00,315
2015-01-01 04:00:00,274
...,...
2016-06-30 19:00:00,401
2016-06-30 20:00:00,390
2016-06-30 21:00:00,460
2016-06-30 22:00:00,405
